# OOP & Standard Python

This notebook is about ensuring your classes play well with standard python approaches.

The approaches in this notebook are referred to as operator overloading. This simply
means adjusting operator comparisons with some *ad hoc* logic, in order to return more
intuitive results. Let's take a look with logicals.

## Equivalence

In [1]:
class Human:
    def __init__(self, name, iq=100):
        self.name = name
        self.iq = iq

In [3]:
reg = Human("Reginald", 120)
fran = Human("Francine", 121)
print(type(reg))
print(type(fran))

<class '__main__.Human'>
<class '__main__.Human'>


In [5]:
fran == reg
# this seems OK

False

In [6]:
# This doesn't seem right
reg1 = Human("Reginald", 120)
reg == reg1

False

This is because by default, python compares the object references rather than the data.
To overcome this, we need to customise what should happen when 2 classes are compared. 
This is achieved with the comparator `__eq__()`.

In [7]:
class Human:
    def __init__(self, name, iq=100):
        self.name = name
        self.iq = iq
    
    def __eq__(self, other):
        return (self.name == other.name) and (self.iq == other.iq)
reg = Human("Reginald", 120)
fran = Human("Francine", 121)
reg1 = Human("Reginald", 120)


In [8]:
print(reg == fran)
print(reg == reg1)

False
True


This seems more intuitive. However, we should ensure we check the class too! Think about
the following case where we check a human against an android. 

In [10]:
class Android(Human):
    pass
reg2 = Android("Reginald", 120)
print(reg2 == reg)
# Uh oh!

True


The equivalence is simply based upon the data we told it to compare - name and IQ. It's 
best to ensure that the class is also checked. Let's update the Human class and re-run
the comparison.

In [11]:
class Human:
    def __init__(self, name, iq=100):
        self.name = name
        self.iq = iq
    
    def __eq__(self, other):
        return (self.name == other.name) and (self.iq == other.iq) and \
        (type(self) == type(other))

class Android(Human):
    pass

fran = Human("Francine", 121)
fran1 = Android("Francine", 121)
print(fran == fran1)
# Phew - we can now distinguish between a human and an android.


False


***

## Representing an object

There are 2 main approaches considered:

* `__str__()` for responding to `print()` or `str()` statements.
* `__repr__()` as a fallback for `str()` or implicit representation of an object. Also
for `repr()` statements.

`str()` calls expect more user friendly information about the object and you can decide
how best to represent the important info.

`repr()` expects to show code that could be used to **reproduce** the object, so ensure
that it is valid.



In [12]:
str(fran)
# not very informative. Let's update the class.

'<__main__.Human object at 0x7fac2807edd0>'

In [18]:
class Human:
    def __init__(self, name, iq=100):
        self.name = name
        self.iq = iq
    
    def __eq__(self, other):
        return (self.name == other.name) and (self.iq == other.iq) and \
        (type(self) == type(other))
    
    def __str__(self):
        s = """
        {t} name: {n}
        {t} iq: {iq}
        """.format(t = type(self), n=self.name, iq=self.iq)
        return s

class Android(Human):
    pass
fran = Human("Francine", 121)
fran1 = Android("Francine", 121)

In [22]:
print(fran)
print(fran1)
# a bit better. Now how about repr...


        <class '__main__.Human'> name: Francine
        <class '__main__.Human'> iq: 121
        

        <class '__main__.Android'> name: Francine
        <class '__main__.Android'> iq: 121
        


'<__main__.Human object at 0x7fac18063bb0>'

In [34]:
class Human:
    def __init__(self, name, iq=100):
        self.name = name
        self.iq = iq
    
    def __eq__(self, other):
        return (self.name == other.name) and (self.iq == other.iq) and \
        (type(self) == type(other))
    
    def __str__(self):
        s = """
        {t} name: {n}
        {t} iq: {iq}
        """.format(t = type(self), n=self.name, iq=self.iq)
        return s
    
    def __repr__(self):
        # clean out the class
        c = str(type(self)).split(".")[-1][0:-2]
        return f'{c}("{self.name}", {self.iq})'

class Android(Human):
    pass
fran = Human("Francine", 121)
fran1 = Android("Francine", 121)
print(repr(fran))
repr(fran1)

Human("Francine", 121)


'Android("Francine", 121)'

***

## Exceptions

Exceptions are simply classes. The base python `Exception` class is used to produce the
Exeptions we know and love. We can inherit from them too in order to create our own
custom exception classes.

To start, just create your own empty class from one of the pre-existing python
exceptions that best matches your purposes.


In [36]:
class NameLengthError(Exception): pass
class IQError(NameLengthError): pass # a child of our firstcustom error

class Human:
    # some helpful class attributes
    MIN_NAME_LEN = 1
    MIN_IQ = 1

    def __init__(self, name, iq=100):
        self.name = name
        self.iq = iq
        if len(name) < Human.MIN_NAME_LEN:
            raise NameLengthError("Name is too short!")
        if iq < Human.MIN_IQ:
            raise IQError("iq is too low!")
    
    def __eq__(self, other):
        return (self.name == other.name) and (self.iq == other.iq) and \
        (type(self) == type(other))
    
    def __str__(self):
        s = """
        {t} name: {n}
        {t} iq: {iq}
        """.format(t = type(self), n=self.name, iq=self.iq)
        return s
    
    def __repr__(self):
        # clean out the class
        c = str(type(self)).split(".")[-1][0:-2]
        return f'{c}("{self.name}", {self.iq})'




In [38]:
try:
    Human("", 100)
except NameLengthError:
    print("Caught the NameLengthError")

Caught the NameLengthError


In [41]:
try:
    Human("Alberto", 0)
except IQError:
    print("Caught the IQError")

Caught the IQError


In [40]:
try:
    Human("Alberto", 0)
except NameLengthError:
    print("Caught the child error with the parent!!!")

Caught the child error with the parent!!!


In [48]:
print("Can a child error catch a parent error?")
try:
    Human("", 0)
except IQError:
    print("This doesn't work.")
except NameLengthError:
    print("Nope!")
finally:
    print("So make sure to test child class exceptions first...")

Can a child error catch a parent error?
Nope!
So make sure to test child class exceptions first...
